### **Análise dos dados**

Segue abaixo a análise dos dados inicialmente fornecidos.

Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import confusion_matrix
import seaborn as sns

import matplotlib.pyplot as plt

Inicializando os dados

In [ ]:
df_resultados = pd.read_csv('../data/RESULTADOS_04_06_2024_full_teste.csv', compression='gzip')
df_resultados2 = pd.read_csv('../data/RESULTADOS_02_03_2024_full_teste.csv', compression='gzip')
df_resultados3 = pd.read_csv('../data/RESULTADOS_06_2023_07_2023_full_teste.csv' ,compression='gzip')
df_falhas = pd.read_csv('../data/FALHAS_04_05_06_2024.csv')

Checando os dados

In [ ]:
# Remover coluna errada
df_resultados = df_resultados.drop(columns=['Unnamed: 0'], axis=1)
df_resultados2 = df_resultados2.drop(columns=['Unnamed: 0'], axis=1)
df_resultados3 = df_resultados3.drop(columns=['Unnamed: 0'], axis=1)



In [ ]:
df_falhas = df_falhas.drop(columns=['Unnamed: 0'], axis=1)

Arrumando os dados de falha pois o nome da coluna ficou nas primeiras linhas

In [ ]:
df_falhas.columns = df_falhas.iloc[0]

df_falhas = df_falhas.drop(df_falhas.index[0])

Checar ambos DFs

In [ ]:
df_falhas.head()

Analisando os tipos de dados de falhas

In [ ]:
df_falhas.describe()

In [ ]:
df_resultados.head(20)

### Análise de Dados para Modelo de Falhas em Veículos

Após a análise dos dados, foram feitas as seguintes seleções para o treinamento e teste do modelo:

#### Falhas
- **Colunas a serem utilizadas:**
  - `KNR`: Identificação do veículo.
  - `FALHA`: Indicação da ocorrência da falha.
- **Motivo:** As demais colunas são únicas para os veículos que tiveram falhas e não estão presentes na tabela de `RESULTADOS`. Incluir essas colunas poderia causar viés no modelo.

#### Resultados
- **Colunas a serem descartadas:**
  - `UNIT`
  - `VALUE_ID`
  - `VALUE`
- **Motivo:** Essas colunas não serão utilizadas na análise inicial, mas podem ser incluídas futuramente para melhorar a acurácia e o recall do modelo.

#### Observações Futuras
- **Coluna `Data` da tabela FALHAS:** Pode ser explorada futuramente para criar novas features.

Essa abordagem foi adotada para garantir que o modelo seja treinado e testado com dados que não introduzam viés, permitindo uma análise mais objetiva e confiável.


## Preparação dos dados

Inicialmente, os dados serão integrados em uma única tabela

In [ ]:
# Remoção das linhas com NaN

df_falhas = df_falhas.dropna()
df_resultados = df_resultados.dropna()
df_resultados2 = df_resultados2.dropna()
df_resultados3 = df_resultados3.dropna()

In [ ]:
df_resultados.head()

Convertendo a coluna DATA para um formato melhor

In [ ]:
df_resultados.info()

In [ ]:
# Salvando os dados em "parquet" é mais rápido para leitura

df_resultados.to_parquet('../data/df_resultados.parquet', index=False)
df_resultados2.to_parquet('../data/df_resultados2.parquet', index=False)
df_resultados3.to_parquet('../data/df_resultados3.parquet', index=False)

In [ ]:
# Limpando colunas que não serão usadas agora

colunas = ['UNIT', 'VALUE_ID', 'VALUE']

df_resultados2 = df_resultados2.drop(columns=colunas, axis=1)
df_resultados = df_resultados.drop(columns=colunas, axis=1)
df_resultados3 = df_resultados3.drop(columns=colunas, axis=1)

In [ ]:
# Transformando os DFs de resultados em apenas um DF de resultados maior

df = pd.concat([df_resultados, df_resultados2, df_resultados3])

In [ ]:
# Transformando data em datetime do pandas

df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce')

Segue abaixo o código para a preparação dos dados

In [ ]:
def aggregate_by_id(df, id_value):
    subset = df[df['ID'] == id_value]
    return subset.groupby('KNR').agg(
        NAME=('NAME', 'count'),  # Conta total de NAME
        SOK=('STATUS', lambda x: (x == 10).sum()),
        SNOK=('STATUS', lambda x: (x == 13).sum()),
        DATA=('DATA', lambda x: (x.max() - x.min()).total_seconds() / (3600*24))
    ).rename(columns={
        'NAME': f'ID{id_value}NAME',
        'SOK': f'ID{id_value}SOK',
        'SNOK': f'ID{id_value}SNOK',
        'DATA': f'ID{id_value}DATA'
    })

# Agregando para cada ID
id1 = aggregate_by_id(df, 1)
id2 = aggregate_by_id(df, 2)
id718 = aggregate_by_id(df, 718)

# Combinando os resultados em um único DataFrame
final_df = id1.join(id2, on='KNR', how='outer').join(id718, on='KNR', how='outer').reset_index()

# Reordenando as colunas para o formato desejado
final_df = final_df[['KNR','ID1NAME', 'ID1SOK', 'ID1SNOK', 'ID1DATA', 'ID2NAME', 'ID2SOK', 'ID2SNOK', 'ID2DATA', 'ID718NAME', 'ID718SOK', 'ID718SNOK', 'ID718DATA']]

# Exibindo o resultado
print(final_df)

In [ ]:
# Colocar 0 no lugar de NaN para evitar problemas
final_df = final_df.fillna(0)

In [ ]:
final_df.head(10)

In [ ]:
# Checar o tamanho do DataFrame

final_df.shape

Arrumando o DF de Falhas


In [ ]:
# Deixa todas as falhas em letra maiuscula
df_falhas['FALHA'] = df_falhas['FALHA'].str.upper()

In [ ]:
# Checa o DataFrame
df_falhas.head()

In [ ]:
# Removendo as colunas que não serão utilizadas na predição

colunas_to_remove = ['MODELO', 'COR', 'MOTOR', 'ESTACAO', 'USUARIO', 'HALLE', 'DATA']

df_falhas = df_falhas.drop(columns=colunas_to_remove, axis=1)

In [ ]:
# Remove todas as linhas com KNR repetido
df_falhas_unique = df_falhas.drop_duplicates(subset=['KNR'])

# Checa o DataFrame
df_falhas_unique.head()


In [ ]:
# Muda os valores da coluna de Falha para 1
df_falhas_unique['FALHA'] = 1

In [ ]:
# Checa o DataFrame
df_falhas_unique.head()

Dando merge do Falhas com Resultado

In [ ]:
# Realizando o merge dos dataframes df_falhas e final_df com base na coluna 'KNR'
merged_df = pd.merge(final_df, df_falhas_unique, on='KNR', how='left')

In [ ]:
# Analisa o tamanho do DF final
merged_df.shape

In [ ]:
# Adiciona 0 em todos os NaN
merged_df = merged_df.fillna(0)

In [ ]:
merged_df.head()

Normalização e treinamento do modelo

In [ ]:
# Selecionando apenas as colunas específicas para normalização
cols_to_normalize = ['ID1NAME','ID1SOK', 'ID1SNOK', 'ID1DATA', 'ID2NAME', 'ID2SOK', 'ID2SNOK', 'ID2DATA', 'ID718NAME', 'ID718SOK', 'ID718SNOK', 'ID718DATA']

# Inicializando o MinMaxScaler
scaler = MinMaxScaler()

# Aplicando a normalização
merged_df[cols_to_normalize] = scaler.fit_transform(merged_df[cols_to_normalize])

# Exibindo o dataframe normalizado
merged_df.head()

In [ ]:
# Separando as features (X) e o target (y)
X = merged_df.drop(columns=['FALHA', 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
y = merged_df['FALHA'] 

In [ ]:
# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class AutoLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(AutoLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Definição da camada LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Camada fully connected para mapear a saída da LSTM para o espaço de saída desejado
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Inicializar o estado oculto e a célula da LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Passar os dados pela LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Mapear a última saída da LSTM para o espaço de saída desejado
        out = self.fc(out[:, -1, :])
        
        return out

In [ ]:
# Definições de hiperparâmetros
input_size = 1  # Tamanho do vetor de entrada (dimensão de cada ponto temporal)
hidden_size = 128  # Número de unidades na camada LSTM
num_layers = 2  # Número de camadas LSTM
output_size = 1  # Tamanho do vetor de saída
num_epochs = 100
learning_rate = 0.001

# Criar instância do modelo
model = AutoLSTM(input_size, hidden_size, num_layers, output_size)

# Definir função de perda e otimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Dados de exemplo (substitua com seus próprios dados)
train_data = np.sin(np.linspace(0, 100, 1000))  # Sinal de senoide como exemplo
train_data = train_data.reshape(-1, 1)

# Converter dados para tensor PyTorch
train_data = torch.tensor(train_data, dtype=torch.float32)

# Treinamento
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(train_data)
    loss = criterion(outputs, train_data)
    
    # Backward pass e otimização
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
model.eval()
with torch.no_grad():
    predicted = model(train_data).cpu().numpy()

plt.plot(train_data.numpy(), label='Original Data')
plt.plot(predicted, label='Predicted Data')
plt.legend()
plt.show()
